In [1]:

from datetime import datetime

# current date and time
now = datetime.now()

str(now)

In [8]:
# import glob
# import os

# list_of_files = glob.glob('data/scrapped_data.csv') # * means all if need specific format then *.csv
# latest_file = max(list_of_files, key=os.path.getctime)
# print latest_file

import os
import platform

def creation_date(path_to_file):

    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime

In [12]:
print(int(creation_date('data/scrapped_data.csv')))

1599973695


In [5]:
import os.path as osp
import os.path as osp
import glob

In [ ]:
os.rename(
    "data/scrapped_data.csv",
    f"data/previous_data/scrapped_data_{creation_date('data/scrapped_data.csv')}.csv"
    )

In [ ]:
def creation_date(path_to_file):

    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime

def move_data():
    os.rename(
    "data/scrapped_data.csv",
    f"data/previous_data/scrapped_data_{creation_date('data/scrapped_data.csv')}.csv"
    )

In [64]:
import pandas as pd
import numpy as np
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

# Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer


# Model saving
import pickle

# Math function needed for models
from math import sqrt

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MinMaxScaler

from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from sklearn.feature_extraction.text import TfidfVectorizer

# --- NEW IMPORT ---
from sklearn.model_selection import GridSearchCV
import time


In [61]:
def kfold_cross_validation(df, k = 10, target = 'category'): 
    # --- KFOLD & DATA ---

    kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 42)
    features = df[[col for col in df if col != target]]
    target = df[target]   

    # --- CREATING MODELS ---

    def create_models():
        models = list()

        models.append(LogisticRegression(solver = 'lbfgs'))
        models.append(KNeighborsClassifier(n_neighbors = 3)) # k = 5 by default
        models.append(MultinomialNB())
        models.append(RandomForestClassifier(max_depth=10, random_state=42))

        return models

    # --- CROSS_VALIDATION ---

    def cv_eval_model(model, cv, X, y):

        scoring = {
            'R2': make_scorer(r2_score),
            'MSE': make_scorer(mean_squared_error),
            'MAE': make_scorer(mean_absolute_error),
            'Accuracy': make_scorer(accuracy_score),
            'Balanced_Acc': make_scorer(balanced_accuracy_score),
            'Precision': make_scorer(precision_score, average = 'macro'),
            'Recall': make_scorer(recall_score, average = 'macro'),
            'F1': make_scorer(f1_score, average = 'macro'),
        }

        cross_val_obj = cross_validate(model, X, y, scoring = scoring, cv = cv, n_jobs = -1, return_estimator = True)  

        return cross_val_obj # trained_model, 

    # --- RUNNING CROSS-VALIDATION ---

    def run_cross_val(models, cv = kfold, X = features, y = target):
        dict_df = dict()

        for model in models:
            cv_mean = cv_eval_model(model, cv, X, y) #trained_model, 

            # with open (type(model).__name__, 'wb') as f:
            #     pickle.dump(model,f) 

            dict_df[type(model).__name__] = cv_mean

        return dict_df

    models_ = create_models()
    dict_df = run_cross_val(models = models_)
    return dict_df

In [106]:
def best_model(df, k = 10, target = 'category'):
    # --- KFOLD & DATA ---

    kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 42)
    features = df[[col for col in df if col != target]]
    target = df[target]  

    # --- CREATE MODELS ---

    def create_models():
        models = list()

        models.append(LogisticRegression(solver = 'lbfgs'))
        models.append(KNeighborsClassifier()) # k = 5 by default
        models.append(MultinomialNB())
        models.append(RandomForestClassifier(random_state=42))

        return models

    # --- SCORES ---

    scoring = {
    'R2': make_scorer(r2_score),
    'MSE': make_scorer(mean_squared_error),
    'MAE': make_scorer(mean_absolute_error),
    'Accuracy': make_scorer(accuracy_score),
    'Balanced_Acc': make_scorer(balanced_accuracy_score),
    'Precision': make_scorer(precision_score, average = 'macro'),
    'Recall': make_scorer(recall_score, average = 'macro'),
    'F1': make_scorer(f1_score, average = 'macro'),
    }

    # --- PARAMETERS ---
        # LR
    Cs = [0.001, 0.01, 0.1, 0.3, 1, 3, 10, 100]

        # KNN
    n_neighbors_ = [3, 5, 8]
        # MultiNB
    alphas = [0, 0.5, 1.0]
        # RandomForest
    n_estimators_ = [50, 100, 150]
    max_features_ = ['sqrt', 'log2']
    ccp_alphas = [0, 0.5, 1]

    dict_param_grid = {
        'LogisticRegression': {'C': Cs},
        'KNeighborsClassifier': {'n_neighbors': n_neighbors_},
        'MultinomialNB': {'alpha': alphas},
        'RandomForestClassifier': {'n_estimators': n_estimators_, 'max_features': max_features_, 'ccp_alpha': ccp_alphas}
    }

    # --- RUNNING GRIDSEARCHCV
    models = create_models()

    df_models = pd.DataFrame(columns = ['model_name', 'best_params', 'best_score', 'best_model'])

    # --- PROGRESS BAR ---
    my_bar = st.progress(0)

    for i, model in enumerate(models):
        param_grid = dict_param_grid[type(model).__name__]
        grid_search = GridSearchCV(
            model,
            param_grid,
            cv = kfold,
            scoring = scoring,
            refit = 'Accuracy',
            n_jobs = -1
        )
        grid_search.fit(features, target)

        df_input = pd.DataFrame({'model_name':f'{type(model).__name__}', 'best_params':grid_search.best_params_, 'best_score':grid_search.best_score_, 'best_model': grid_search.best_estimator_})
        df_models = pd.concat([df_models,df_input], axis = 0, sort = False).reset_index(drop = True)

        my_bar.progress((i+1)/len(models))

    return df_models



In [30]:
def over_under_sampling(df):

    list_count = Counter(df['category']).most_common()
    mid_num = int(((list_count[0][1]-list_count[9][1])/2))
    mid_num = mid_num+list_count[9][1]
    num_samples = mid_num
    strategy = dict(Counter(df['category']))
    
    for i in range(len(strategy)):
        if strategy[i] < num_samples:
            strategy[i] = num_samples

    ros = RandomOverSampler(sampling_strategy=strategy, random_state=42)
    rus = RandomUnderSampler(random_state=42)

    X = df.drop(['category'], axis=1)
    y = df[['category']]

    X_ros, y_ros = ros.fit_resample(X, y)
    X_rous, y_rous = rus.fit_resample(X_ros, y_ros)
    o_u_sample = X_rous.join(y_rous)

    return o_u_sample 

In [20]:
def convert_to_tfidf(df, case_col = 'case', target_col = 'category'):
    
    tfidf = TfidfVectorizer()
    word_count_vectors = tfidf.fit_transform(df[case_col].values).todense().tolist()
    
    features = pd.DataFrame(
    data = word_count_vectors,
    columns = tfidf.get_feature_names()
    )

    df_ = features.merge(df[target_col], left_index=True, right_index= True)
    
    return df_

In [21]:
df_ = pd.read_csv('../data/data.csv')

In [22]:
df_test = df_.drop(columns='Unnamed: 0')

In [26]:
def category_replacer(df, col = 'category', mul = True, main_cat = "Deadbeats"):

    if mul == True: #--- MULTILABEL ---
        dic_cat = {}
        for i, cat in enumerate(list(df[col].unique())):
            dic_cat[cat] = i

    else: #--- CATEGORY VS. NOT CATEGORY ---
        dic_cat = {
            "Deadbeats" : 0,
            'Dunces' : 0,
            'Criminals' : 0,
            'Racists' : 0,
            'Homophobes' : 0,
            'Sexist' : 0,
            'Frenemies' : 0,
            'Cryptic' : 0,
            'Ingrates' : 0,
            'Chaotic Good' : 0
        }
        
        dic_cat[main_cat]  =  1

    df[col].replace(to_replace = dic_cat, inplace = True)
    
    return df, dic_cat

In [33]:
rep_df = category_replacer(df_test)
o_u_df = over_under_sampling(rep_df[0])

In [37]:
final_df = convert_to_tfidf(o_u_df)

In [62]:
kfold_cross_validation(final_df)

{'LogisticRegression': {'fit_time': array([3.14393902, 2.8588872 , 3.09740496, 2.89498305, 3.10038233,
         2.99936795, 3.17797208, 3.09765196, 1.95441699, 1.96075416]),
  'score_time': array([0.18450785, 0.27821994, 0.24474216, 0.26767182, 0.24511385,
         0.18505383, 0.18930578, 0.16417718, 0.08030701, 0.05656672]),
  'estimator': (LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression(),
   LogisticRegression()),
  'test_R2': array([0.76262829, 0.48569463, 0.8595703 , 0.75888486, 0.71316819,
         0.70273794, 0.88341685, 0.95230689, 0.93406341, 0.66504214]),
  'test_MSE': array([1.95652174, 4.23913043, 1.15217391, 1.97826087, 2.39130435,
         2.47826087, 0.95652174, 0.39130435, 0.54347826, 2.76086957]),
  'test_MAE': array([0.47826087, 0.7173913 , 0.36956522, 0.5       , 0.52173913,
         0.60869565, 0.2

In [63]:
pd.DataFrame(kfold_cross_validation(final_df)['LogisticRegression'])

,fit_time,score_time,estimator,test_R2,test_MSE,test_MAE,test_Accuracy,test_Balanced_Acc,test_Precision,test_Recall,test_F1
0,3.943365,0.344837,LogisticRegression(),0.762628,1.956522,0.478261,0.869565,0.870,0.867857,0.870,0.858117
1,4.003656,0.301153,LogisticRegression(),0.485695,4.239130,0.717391,0.869565,0.875,0.897500,0.875,0.867399
2,4.248466,0.204503,LogisticRegression(),0.859570,1.152174,0.369565,0.826087,0.825,0.793333,0.825,0.803810
3,4.208723,0.167869,LogisticRegression(),0.758885,1.978261,0.500000,0.847826,0.850,0.797143,0.850,0.817172
4,4.026634,0.262611,LogisticRegression(),0.713168,2.391304,0.521739,0.869565,0.880,0.897143,0.880,0.879444
5,3.948291,0.221415,LogisticRegression(),0.702738,2.478261,0.608696,0.804348,0.815,0.839524,0.815,0.818038
6,3.933197,0.225026,LogisticRegression(),0.883417,0.956522,0.260870,0.891304,0.895,0.910000,0.895,0.895000
7,3.971508,0.262017,LogisticRegression(),0.952307,0.391304,0.130435,0.934783,0.935,0.943333,0.935,0.935354
8,2.146743,0.113792,LogisticRegression(),0.934063,0.543478,0.152174,0.956522,0.950,0.966667,0.950,0.953247
9,2.114204,0.094597,LogisticRegression(),0.665042,2.760870,0.673913,0.826087,0.830,0.838095,0.830,0.825505


In [105]:
best_model(final_df)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [97]:
df = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                   'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})


other = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'a'],
                      'B': ['B0', 'B1', 'B2', 'b']})

print(pd.concat([df,other], axis = 0, sort = False).reset_index(drop = True))

key    A    B
0  K0   A0  NaN
1  K1   A1  NaN
2  K2   A2  NaN
3  K3   A3  NaN
4  K4   A4  NaN
5  K5   A5  NaN
6  K0  NaN   B0
7  K1  NaN   B1
8  K2  NaN   B2
9   a  NaN    b
